# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()


load_dotenv(override=True)
api_key = os.getenv('DEEPSEEK_API_KEY')

if api_key and api_key.startswith('sk-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'deepseek-chat'
openai = OpenAI(api_key=api_key,base_url="https://api.deepseek.com/v1") 

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nanonets/Nanonets-OCR-s',
 '/MiniMaxAI/MiniMax-M1-80k',
 '/Menlo/Jan-nano',
 '/tencent/Hunyuan3D-2.1',
 '/moonshotai/Kimi-Dev-72B',
 '/models',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/enzostvs/deepsite',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/MiniMaxAI/MiniMax-M1',
 '/spaces/nvidia/PartPacker',
 '/spaces',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Nemotron-Personas',
 '/datasets/nvidia/AceReason-1.1-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'discord page', 'url': 'https://huggingface.co/join/discord'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discord page', 'url': 'https://huggingface.co/join/discord'}]}
Landing page:
Webpage Title:
Hugging Face 

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discord page', 'url': 'https://huggingface.co/join/discord'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnanonets/Nanonets-OCR-s\nUpdated\n1 day ago\n•\n125k\n•\n1k\nMiniMaxAI/MiniMax-M1-80k\nUpdated\n4 days ago\n•\n7.59k\n•\n472\nMenlo/Jan-nano\nUpdated\n5 days ago\n•\n28.5k\n•\n340\ntencent/Hunyuan3D-2.1\nUpdated\n2 days ago\n•\n18.2k\n•\n403\nmoonshotai/Kimi-Dev-72B\nUpdated\n5 days ago\n•\n8.42k\n•\n237\nBrowse 1M+ models\nSpaces\nRunning\n621\n621\nSparc3D\n🏃\nNext-Gen High-Resolution 3D

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'join', 'url': 'https://huggingface.co/join'}, {'type': 'brand', 'url': 'https://huggingface.co/brand'}, {'type': 'learn', 'url': 'https://huggingface.co/learn'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discord', 'url': 'https://huggingface.co/join/discord'}]}


```markdown
# 🌟 Hugging Face – The AI Community Building the Future

## 🚀 **Who We Are**  
Hugging Face is the leading open platform for **machine learning collaboration**, where the global ML community builds, shares, and deploys cutting-edge AI models, datasets, and applications.  

### **Our Mission**  
To democratize AI by empowering developers, researchers, and enterprises with:  
- **1M+ open models** (text, image, video, 3D, and more)  
- **400k+ AI applications** (Spaces)  
- **250k+ datasets**  

---

## 🏆 **Why Hugging Face?**  

### **For Developers & Researchers**  
- **Collaborate**: Host and share public/private models, datasets, and apps.  
- **Build Faster**: Leverage open-source tools like Transformers, Diffusers, and Tokenizers.  
- **Showcase Work**: Grow your ML portfolio in a community of 50k+ organizations.  

### **For Enterprises**  
- **Enterprise Hub**: Secure AI development with SSO, audit logs, and private datasets.  
- **Scalable Compute**: Deploy models on GPU-powered **Inference Endpoints** (from $0.60/hour).  
- **Trusted by**: Google, Meta, Microsoft, Intel, Grammarly, and more.  

---

## 💡 **Key Offerings**  

### **Open-Source Tools**  
- **Transformers**: 145k+ stars – SOTA models for PyTorch/TensorFlow.  
- **Diffusers**: 29k+ stars – Diffusion models for generative AI.  
- **TRL & PEFT**: Optimized LLM training/finetuning.  

### **Community & Growth**  
- **Trending Models**: Discover breakthroughs like Nanonets-OCR or Hunyuan3D-2.1.  
- **Spaces**: Deploy apps (e.g., 3D generation, code synthesis) with ease.  

---

## 🏢 **Company & Culture**  
- **Open Source First**: Built by and for the ML community.  
- **Inclusive Collaboration**: Join forums, Discord, and GitHub discussions.  
- **Careers**: Work on foundational AI tools – [Explore Jobs](https://huggingface.co/jobs).  

---

## 📌 **Get Started**  
- **Free Signup**: [Join the community](https://huggingface.co)  
- **Enterprise Plans**: From **$20/user/month** (priority support, security).  

🔗 *Website*: [huggingface.co](https://huggingface.co) | *GitHub*: [github.com/huggingface](https://github.com/huggingface)  

---

*"The Home of Machine Learning" – Where the future of AI is built, together.*  
``` 

This brochure highlights Hugging Face’s **community-driven ethos**, **technical offerings**, and **enterprise solutions** while emphasizing its role as a hub for innovation. Let me know if you'd like adjustments!

In [21]:
create_brochure("tencent", "https://www.tencent.com/zh-cn/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.tencent.com/zh-cn/about.html'}, {'type': 'business page', 'url': 'https://www.tencent.com/zh-cn/business.html'}, {'type': 'employees page', 'url': 'https://www.tencent.com/zh-cn/employees.html'}, {'type': 'esg page', 'url': 'https://www.tencent.com/zh-cn/esg.html'}, {'type': 'investors page', 'url': 'https://www.tencent.com/zh-cn/investors.html'}, {'type': 'media page', 'url': 'https://www.tencent.com/zh-cn/media.html'}, {'type': 'careers page', 'url': 'https://careers.tencent.com/'}, {'type': 'join page', 'url': 'https://join.qq.com/'}]}


```markdown
# Tencent 腾讯  
**Connecting People, Empowering Businesses, Shaping the Future**  

## About Tencent  
Founded in 1998 and headquartered in Shenzhen, China, **Tencent** is a global leader in internet and technology, dedicated to improving lives through innovative products and services. With a mission of **"User-Oriented, Tech for Good,"** Tencent connects over 1 billion people worldwide via its platforms while driving digital transformation across industries.  

### Key Highlights:  
- **Public Listing:** Listed on the Hong Kong Stock Exchange (2004).  
- **Core Values:** User-centric innovation, social responsibility, and sustainable growth.  
- **Global Impact:** Powering communication (WeChat/QQ), entertainment (gaming, digital content), fintech (WeChat Pay), and cloud solutions (Tencent Cloud).  

---

## Business Segments  
Tencent operates through six core business groups:  

1. **CDG (Corporate Development Group)**  
   - Fintech, advertising, and international expansion.  
2. **CSIG (Cloud & Smart Industries Group)**  
   - AI, cloud computing, and smart industry solutions (retail, healthcare, education).  
3. **IEG (Interactive Entertainment Group)**  
   - Gaming, eSports, and interactive content.  
4. **PCG (Platform & Content Group)**  
   - Social platforms (QQ), content ecosystems (video, news, sports), and IP development.  
5. **TEG (Technology Engineering Group)**  
   - Infrastructure, R&D, and technical support.  
6. **WXG (Weixin Group)**  
   - WeChat ecosystem (payments, mini-programs, enterprise solutions).  

**Innovation Focus:** AI, IoT, multimedia tech, and sustainability (e.g., carbon neutrality by 2030).  

---

## Culture & Careers  
### Workplace & Growth  
- **Learning:** "Tencent Academy" for continuous skill development.  
- **Environment:** Collaborative spaces, employee activities, and global offices.  
- **ESG Commitment:** Initiatives like wildlife conservation (AI for snow leopard protection), "Games for Earth" alliance, and Great Wall preservation.  

### Join Us  
- **Hiring Programs:** Campus recruitment, global internships (e.g., 2025 program), and specialized roles in tech, gaming, and cloud.  
- **Culture:** Fast-paced, inclusive, and mission-driven—ideal for innovators passionate about tech’s societal impact.  

---

## Recent Milestones (2025)  
- Launched AI model **"Species Eye"** for wildlife conservation.  
- Joined **"Playing for the Planet"** alliance to promote green gaming.  
- Enhanced cross-border payment solutions for international travelers via **WeChat Pay**.  

---

**Connect With Us**  
- [Careers](https://careers.tencent.com) | [Investors](https://investors.tencent.com) | [Media](https://news.tencent.com)  
- Follow: `WeChat Official Accounts | LinkedIn | Twitter`  

*"Tech for Good" isn’t just a slogan—it’s our blueprint for the future.*  

© 1998–2025 Tencent. All rights reserved.  
```  

This brochure highlights Tencent’s **business diversity**, **culture of innovation**, and **social impact**, tailored for stakeholders seeking a concise yet comprehensive overview. Let me know if you'd like to emphasize specific areas!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>